Data File ==> DataFrame, Numpy (전처리) ==> Tensor ==> Dataset(피처 + 타겟) ==> DataLoader 생성

### Dataset & DataLoader 살펴보기
- Pytorch에서 배치크기만 데이터를 조절하기 위한 메카니즘
- Dataset : 사용 데이터를 기반으로 사용자 정의 클래스 작성
- DataLoader : 지정된 Dataset에서 지정된 batch size만큼 피처와 타겟을 추출하여 전달

[1] 모듈로딩 및 데이터 준비   (원래는 전처리 까지 해야하는데! 여기서늕 생략했다)

In [298]:
### ===> 모듈 로딩
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd

[2] 데이터셋 생성

- 사용자 정의 데이터셋 생성

In [299]:
file = '../data/iris.csv'
irisDF = pd.read_csv(file, header = None)
irisDF.columns = ['sepal_length', 'sepal_width', 'petal_length','petal_width','variety']

In [300]:
irisNP = np.loadtxt(file, delimiter = ',', usecols = [0,1,2,3])

In [301]:
# 데이터의 타입 체크
type(irisDF), type(irisNP), irisDF.__class__.__name__, irisNP.__class__.__name__

(pandas.core.frame.DataFrame, numpy.ndarray, 'DataFrame', 'ndarray')

In [302]:
if irisDF.__class__.__name__ == 'DataFrame' :
    print('DF')
else:
    print('-----')

DF


In [303]:
print(isinstance(irisDF, pd.DataFrame), isinstance(irisNP, pd.DataFrame), isinstance(irisDF, np.ndarray),isinstance(irisNP, np.ndarray), sep = '\n')

True
False
False
True


In [304]:
### 사용자 정의 DataSet 클래스
# - 데이터의 Tensor 변환

class DLDataset(Dataset):
    # 초기화 함수 콜백함수 (callback function)
    def __init__(self, x_data, y_data):
        super().__init__()
        
        # x,y 데이터 ==> ndarray
        x_data = x_data.values if isinstance(x_data, pd.DataFrame) else x_data
        y_data = y_data.values if isinstance(y_data, pd.DataFrame) else y_data
        # 넘파이면 그대로 ㄱㅊ    다만 데이터프레임이면 values만 뺴서 갖고오자능
        
        # ndarray ==> tensor
        self.feature = torch.FloatTensor(x_data)
        self.target = torch.LongTensor(y_data)         # 라벨 인코딩 형식으로 진행함 (원핫인코딩 ㄴㄴ)
        
    # 데이터셋의 개수 체크 함수 콜백함수 (callback function)
    def __len__(self):
        return self.target.shape[0]
    
    # 특정 인덱스 데이터 + 라벨 반환 콜백함수 (callback function)
    def __getitem__(self, index):
        return self.feature[index], self.target[index]


In [305]:
## 피처와 라벨로 분리
featureDF = irisDF[irisDF.columns[:-1]]
targetDF = irisDF[irisDF.columns[-1]]

print(f"featureDF => {featureDF.shape} , {featureDF.ndim}D")
print(f"targetDF => {targetDF.shape} , {targetDF.ndim}D")

featureDF => (150, 4) , 2D
targetDF => (150,) , 1D


In [306]:
# object 타입 타겟 ===> int 타입 타겟 변환
from sklearn.preprocessing import LabelEncoder

targetNP = LabelEncoder().fit_transform(targetDF)
targetNP = targetNP.reshape(-1,1)
print(targetNP.shape, targetNP.ndim)

(150, 1) 2


In [307]:
# 데이터셋 생성  ==> DF, NP
my_dataset = DLDataset(featureDF, targetNP)

In [308]:
print(my_dataset[0], featureDF.iloc[0], targetDF[0], sep = '\n\n')

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0]))

sepal_length    5.1
sepal_width     3.5
petal_length    1.4
petal_width     0.2
Name: 0, dtype: float64

Iris-setosa


In [309]:
# 데이터셋 생성 => NP, NP
my_dataset2 = DLDataset(irisNP, targetNP)
print(my_dataset2[0])

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0]))


- [2-3] 학습용, 검증용, 테스트용 Dataset <hr>

In [310]:
### ===> 파이토치
from torch.utils.data import random_split

# 학습용, 검증용, 테스트 데이터 비율
seed = torch.Generator().manual_seed(2)

trainDS, validDS, testDS = random_split(my_dataset2, [0.7, 0.1, 0.2], generator = seed )

print(f"trainDS => {len(trainDS)}, validDS => {len(validDS)}, testDS => {len(testDS)}")

trainDS => 105, validDS => 15, testDS => 30


In [311]:
# print(f"Subset 속성 =>\n indices: \n {trainDS.indices} \n dataset : {trainDS.dataset}")

In [312]:
# print(f"Subset 속성 =>\n indices: \n {validDS.indices} \n dataset : {validDS.dataset}")

[3] DataLoader 생성 : 학습용, 검증용, 테스트용 <hr>

In [313]:
# DataLoader 생성
# drop_last 매개변수 : 배치 사이즈로 데이터셋 분리 후 남는 데이터 처리 방법 설정 [기본 : False]
batch = 10
trainDL = DataLoader(trainDS, batch_size = batch)
validDL = DataLoader(validDS, batch_size = batch)
testDL = DataLoader(testDS, batch_size = batch)

len(trainDL), len(validDL), len(testDL)

(11, 2, 3)

In [314]:
# Epoch당 반복 단위
print(f'batch_size : {batch}')
print(f"trainDS => {len(trainDS)}개, validDS => {len(validDS)}개, trainDS => {len(trainDS)}개")
print(f"trainDL => {len(trainDL)}개, validDL => {len(validDL)}개, testDL => {len(testDL)}개")

batch_size : 10
trainDS => 105개, validDS => 15개, trainDS => 105개
trainDL => 11개, validDL => 2개, testDL => 3개


[4] Model 클래스 정의 : 입/출력 피처수, 층 수, 은닉층의 노드수 <hr>
- 구조설계
    * 입력층 : 입력 <= 피처 갯수, iris 4개 
    * 은닉층 : 마음대로 알아서 잘
    * 출력층 : 출력 <= [분류] 타겟 클래스 갯수  [회귀] 1개

In [315]:
# 모델 클래스 정의
# 클래스명 : CModel

class CModel(nn.Module):    # nn.Module을 상속받는다.
    
    # 모델 구성 요소 정의 함수
    def __init__(self, in_, out_):
        super().__init__()
        self.input_layer = nn.Linear(in_, 100)
        self.relu = nn.ReLU()
        self.hidden_layer = nn.Linear(100, 27)
        self.output_layer = nn.Linear(27, out_)
        
    # 순방향 학습 진행
    def forward(self, x):
        x = self.input_layer(x)            # W1x1 + W2x2 _ ... + Wnxn + b 반환
        x = self.relu(x)                   # relu 함수 결과 100개 반환
        x = self.hidden_layer(x)
        x = self.relu(x)
        x = self.output_layer(x)
        return x

[5] 학습 준비 : 실행 디바이스, 모델, 최적화, 손실함수, 학습 횟수, 학습 함수, 평가 함수, 예측 함수 <hr>

In [316]:
# 실행 디바이스 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# 학습 횟수
EPOCHS = 50

In [317]:
# 모델 인스턴스
IN, OUT = my_dataset2.feature.shape[1], len(np.unique(targetDF))
print(IN, OUT)
model = CModel(IN, OUT).to(DEVICE)

4 3


In [318]:
# 손실 함수
LOSS_FN = nn.CrossEntropyLoss().to(DEVICE)

# 최적화 인스턴스
import torch.optim as optim
OPTIMIZER = optim.Adam(model.parameters())

- 학습 및 검증 관련 함수 정의

In [319]:
import torchmetrics.functional as metrics  # 정확도를 위한 모듈

In [320]:
### ===> 학습 진행 함수
def training():
    # 학습 모드 => 정규화, 경사하강법, 드랍아웃 등의 기능 활성화
    model.train()
    
    # 배치 크기만큼 학습 진행 및 저장
    train_loss = []
    for cnt, (feature, target) in enumerate(trainDL):
        # print(cnt, feature, target)
        # 배치 크기만큼의 학습 데이터 준비      
        feature, target = feature.to(DEVICE), target.to(DEVICE)
        
        target = target.squeeze()
        
        # 학습
        pre_target = model(feature)
        # print(f"pre_target => {pre_target.shape}, {pre_target.ndim}D")
        # print(f"target => {target.shape}, {target.ndim}D")
        
        
        # 손실 계산
        loss = LOSS_FN(pre_target, target)
        train_loss.append(loss)
        
        # W, b 업데이트
        OPTIMIZER.zero_grad()
        loss.backward()
        OPTIMIZER.step()
        
        # 배치 단위 학습 진행 메시지 출력
        # print(f"[Train {cnt} batch LOSS] ===> {loss}")
        
        # 정확도 Accuracy
        acc = metrics.accuracy(pre_target, target, task = 'multiclass', num_classes = OUT)
        # print(f" ACC : {acc}")
        
    # 에포크 단위 학습 진행 메시지 출력
    print(f"[Train loss] ===> {loss}")
    # print(f"ACC : {acc}")
    return train_loss

In [321]:
### ===> 검증 및 평가 진행 함수
# 매개변수 dataLoader : 검증 또는 테스트 데이터셋에 대한 Loader
def testing(dataLoader):
    # 추론모드 => 정규화, 경사하강법, 드랍아웃 등의 기능 비활성화
    model.eval()
    
    with torch.no_grad():
        # 배치 크기 만큼 학습 진행 및 저장
        val_loss = []
        for cnt, (feature, target) in enumerate(dataLoader):
            # 배치 크기 만큼의 학습 데이터 준비
            feature, target = feature.to(DEVICE), target.to(DEVICE)
            target = target.squeeze()
            
            # 학습
            pre_target = model(feature)
            
            # 손실 계산
            loss = LOSS_FN(pre_target, target)
            val_loss.append(loss)
           
            
        # 에포크 단위 학습 진행 메시지 출력
        acc = metrics.accuracy(pre_target, target, task = 'multiclass', num_classes = OUT)
        print(f"[Valid loss1] ===> {loss}")
        # print(f"[Valid ACC] : {acc}")
        
        return val_loss

In [322]:
### ===> 예측 함수
def predict():
    pass

[6] 학습 진행 <hr>

In [323]:
### ===> 지정된 횟수만큼 처음부터 ~ 끝까지 학습 및 검증 진행
### ===> 목표 : 최적 (Error 최소화)의 W, b를 가진 모델 완성
### ===> 

from torch.optim.lr_scheduler import ReduceLROnPlateau

# optimizer 초기화 및 모델과 연결
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# ReduceROnPlateau 스케줄러 설정
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=10, verbose=True)


# 학습 중간에 손실 값을 사용하여 스케줄러 업데이트
for eps in range(EPOCHS):
    # ======================= 학습 =======================
    train_loss = training()
    
    # ======================= 검증 =======================
    valid_loss = testing(validDL)
    
    print(f"[{eps}/{EPOCHS}] TRAIN {sum(train_loss)/len(train_loss)} VALID {sum(valid_loss)/len(valid_loss)}\n")
    # print(valid_loss)
    
    
    # ======================= 학습 스케줄러 =======================
    # 조기 종료 기능 ==> 조건 : val)loss가 지정된 횟수 (예 : 5) 이상 개선이 안되면 학습 종료
    
    # 학습률 조절 : ReduceLROnPlateau 스케줄러에 현재 검증 손실값 전달
    scheduler.step(valid_loss[1])
    
    # 조기 종료 체크
    # 여기서 num_bad_epohcs는 연속적으로 손실이 개선되지 않은 epoch의 수
    if scheduler.num_bad_epochs >= scheduler.patience:
        print(f"Early stopping at epoch {eps}")
        break
    

C:\ProgramData\anaconda3\envs\Torch_PY38\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


[Train loss] ===> 1.1688792705535889
[Valid loss1] ===> 0.9410861730575562
[0/50] TRAIN 1.1681147813796997 VALID 1.0178366899490356

[Train loss] ===> 1.0054303407669067
[Valid loss1] ===> 0.8183774948120117
[1/50] TRAIN 0.9926506280899048 VALID 0.863702654838562

[Train loss] ===> 0.8562873005867004
[Valid loss1] ===> 0.6744747757911682
[2/50] TRAIN 0.8463172316551208 VALID 0.6841503381729126

[Train loss] ===> 0.7381733655929565
[Valid loss1] ===> 0.5854506492614746
[3/50] TRAIN 0.7243310809135437 VALID 0.5723443627357483

[Train loss] ===> 0.6553093791007996
[Valid loss1] ===> 0.5020638704299927
[4/50] TRAIN 0.6291073560714722 VALID 0.4794863164424896

[Train loss] ===> 0.5750821232795715
[Valid loss1] ===> 0.465310662984848
[5/50] TRAIN 0.545525312423706 VALID 0.426993191242218

[Train loss] ===> 0.5339286923408508
[Valid loss1] ===> 0.4206434190273285
[6/50] TRAIN 0.49109217524528503 VALID 0.380465030670166

[Train loss] ===> 0.49310868978500366
[Valid loss1] ===> 0.40085133910179

반장이 작성한 학습 스케줄러 코드

In [324]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

# optimizer 초기화 및 모델과 연결
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# ReduceROnPlateau 스케줄러 설정
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=10, verbose=True)


# 학습 중간에 손실 값을 사용하여 스케줄러 업데이트
for eps in range(EPOCHS): # 50
    train_loss=training()
    valid_loss=testing()
    print(f'{eps}번째 반복 ==> train_loss : {train_loss}, valid_loss : {valid_loss}')
    
    # 학습률 조절 : ReduceLROnPlateau 스케줄러에 현재 검증 손실값 전달
    scheduler.step(valid_loss)
    
    # 조기 종료 체크
    # 여기서 num_bad_epohcs는 연속적으로 손실이 개선되지 않은 epoch의 수
    if scheduler.num_bad_epochs > scheduler.patience:
        print(f"Early stopping at epoch {eps}")
        break

[Train loss] ===> 0.2313561737537384


TypeError: testing() missing 1 required positional argument: 'dataLoader'

In [ ]:
# DataLoader 속성
for _ , (feature, target) in enumerate(trainDL):
    print(f"[{_}] feature {feature.shape}")
    ## 로더에서 가지고온 데이터만큼 학습 진행 